In [2]:
!pip uninstall psycopg2
!pip install psycopg2

In [30]:
import psycopg2

from absenteeism_module import *
import config

In [23]:
conn = psycopg2.connect(user = config.DATABASE_CONFIG['user'],
                        password = config.DATABASE_CONFIG['password'],
                        host = config.DATABASE_CONFIG['host'],
                        port = config.DATABASE_CONFIG['port'],
                        database = config.DATABASE_CONFIG['dbname'])

In [43]:
conn.set_session(autocommit=True)

ProgrammingError: set_session cannot be used inside a transaction

In [24]:
cur = conn.cursor()

In [26]:
print (conn.get_dsn_parameters(),"\n")

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'Absenteeism', 'host': '127.0.0.1', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [ ]:
# CREATE TABLE predicted_outputs
# (
# reason_1 BIT NOT NULL,
# reason_2 BIT NOT NULL,
# reason_3 BIT NOT NULL,
# reason_4 BIT NOT NULL,
# month_value INT NOT NULL,
# transportation_expense INT NOT NULL,
# age INT NOT NULL,
# body_mass_index INT NOT NULL,
# education BIT NOT NULL,
# children INT NOT NULL,
# pet INT NOT NULL,
# probability FLOAT NOT NULL,
# prediction BIT NOT NULL
# )

In [28]:
cur.execute("SELECT * FROM predicted_outputs")

cur.fetchall()

[]

In [47]:
preprocessed_data = pd.read_csv('Absenteeism_new_data.csv')
preprocessed_data.head()
model = absenteeism_model('model', 'scaler')
model.load_and_clean_data('Absenteeism_new_data.csv')
df_new_obs = model.predicted_outputs()

df_new_obs.head()

Index(['Month Value', 'Transportation Expense', 'Age', 'Body Mass Index',
       'Children', 'Pet'],
      dtype='object')


C:\Users\TaimurAdil\anaconda3\lib\site-packages\sklearn\base.py:313: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.1 when using version 0.22. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\TaimurAdil\anaconda3\lib\site-packages\sklearn\base.py:313: UserWarning: Trying to unpickle estimator StandardScaler from version 0.19.1 when using version 0.22. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pet,Probability,Prediction
0,0,0.0,0,1,6,179,30,19,1,0,0,0.122799,0
1,1,0.0,0,0,6,361,28,27,0,1,4,0.873392,1
2,0,0.0,0,1,6,155,34,25,0,2,0,0.268305,0
3,0,0.0,0,1,6,179,40,22,1,2,0,0.196385,0
4,1,0.0,0,0,6,155,34,25,0,2,0,0.723502,1


In [48]:
df_new_obs[df_new_obs.columns.values[6]][0]

30

In [53]:
insert_query = 'INSERT INTO predicted_outputs VALUES '

for i in range(df_new_obs.shape[0]):
    insert_query += '(' 
    
    for j in range(df_new_obs.shape[1]):
        if j in (0, 1, 2, 3, 8, 12):
            insert_query += str(df_new_obs[df_new_obs.columns.values[j]][i]) + '::BIT, '
        else:
            insert_query += str(df_new_obs[df_new_obs.columns.values[j]][i]) + ', '
    
    insert_query = insert_query[:-2] + '), ' 

insert_query = insert_query[:-2]
insert_query

'INSERT INTO predicted_outputs VALUES (0::BIT, 0.0::BIT, 0::BIT, 1::BIT, 6, 179, 30, 19, 1::BIT, 0, 0, 0.12279926243494323, 0::BIT), (1::BIT, 0.0::BIT, 0::BIT, 0::BIT, 6, 361, 28, 27, 0::BIT, 1, 4, 0.8733923470447209, 1::BIT), (0::BIT, 0.0::BIT, 0::BIT, 1::BIT, 6, 155, 34, 25, 0::BIT, 2, 0, 0.2683049546034322, 0::BIT), (0::BIT, 0.0::BIT, 0::BIT, 1::BIT, 6, 179, 40, 22, 1::BIT, 2, 0, 0.1963853143922252, 0::BIT), (1::BIT, 0.0::BIT, 0::BIT, 0::BIT, 6, 155, 34, 25, 0::BIT, 2, 0, 0.7235017225544896, 1::BIT), (1::BIT, 0.0::BIT, 0::BIT, 0::BIT, 6, 225, 28, 24, 0::BIT, 1, 2, 0.7168905020985188, 1::BIT), (1::BIT, 0.0::BIT, 0::BIT, 0::BIT, 6, 118, 46, 25, 0::BIT, 2, 0, 0.5705238370679658, 1::BIT), (0::BIT, 0.0::BIT, 0::BIT, 1::BIT, 6, 179, 30, 19, 1::BIT, 0, 0, 0.12279926243494323, 0::BIT), (0::BIT, 0.0::BIT, 0::BIT, 1::BIT, 6, 118, 37, 28, 0::BIT, 0, 0, 0.13411805408968352, 0::BIT), (1::BIT, 0.0::BIT, 0::BIT, 0::BIT, 6, 118, 37, 28, 0::BIT, 0, 0, 0.5250065611752316, 1::BIT), (0::BIT, 0.0::BIT, 

In [55]:
conn.commit()
cur.execute(insert_query)
conn.commit()
conn.close()